In [19]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

import os, sys, progressbar, argparse 
import commentjson as json
from keras.datasets import cifar10
from imutils import paths
from skimage.io import imread
import imageio
import cv2

from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib inline


from sklearn.cross_validation import KFold
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split


from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.models import model_from_json

In [20]:
def error_rate(predictions, labels):
  """Return the error rate and confusions."""
  correct = numpy.sum(numpy.argmax(predictions, 1) == numpy.argmax(labels, 1))
  total = predictions.shape[0]

  error = 100.0 - (100 * float(correct) / float(total))

  confusions = numpy.zeros([10, 10], numpy.float32)
  bundled = zip(numpy.argmax(predictions, 1), numpy.argmax(labels, 1))
  for predicted, actual in bundled:
    confusions[predicted, actual] += 1
    
  return error, confusions

In [21]:

def simpleNN(nb_classes,img_rows,img_cols,img_channels):
    #####Fail
    model = Sequential()
    model.add(Dense(512, input_shape=(img_channels, img_rows, img_cols)))
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
    sgd = SGD(lr=1e-2, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
    return model

def simpleCNN(nb_classes, nb_filters,nb_conv,nb_pool,img_rows,img_cols,img_channels, Dense_Before_Out=12):
    model = Sequential()
    model.add(Convolution2D(nb_filters, nb_conv, nb_conv,
                        border_mode='valid',
                        input_shape=(img_channels, img_rows, img_cols)))
    model.add(Activation('relu'))
    model.add(Convolution2D(nb_filters, nb_conv, nb_conv))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    
    model.add(Dense(Dense_Before_Out))
    model.add(Activation('tanh'))
    
    model.add(Dropout(0.5))
    
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
    sgd = SGD(lr=1e-2, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
    return model


def simpleCNN_2(nb_classes, nb_filters,nb_conv,nb_pool,img_rows,img_cols,img_channels, Dense_Before_Out=32):
    model = Sequential()
    model.add(Convolution2D(nb_filters, nb_conv, nb_conv,
                        border_mode='valid',
                        input_shape=(img_channels, img_rows, img_cols)))
    model.add(Activation('tanh'))
    model.add(Convolution2D(nb_filters, nb_conv, nb_conv))
    model.add(Activation('tanh'))
    model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    
    model.add(Dense(10*Dense_Before_Out, init='lecun_uniform'))
    model.add(Activation('tanh'))    
    
    model.add(Dense(Dense_Before_Out, init='lecun_uniform'))
    model.add(Activation('tanh'))
    
    model.add(Dense(nb_classes, init='lecun_uniform'))
    model.add(Activation('softmax'))
    sgd = SGD(lr=1e-2, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
    return model



def simpleCNN_3(nb_classes, nb_filters,nb_conv,nb_pool,img_rows,img_cols,img_channels, Dense_Before_Out=520):
    model = Sequential()
    model.add(Convolution2D(nb_filters, nb_conv, nb_conv,
                        border_mode='valid',
                        input_shape=(img_channels, img_rows, img_cols)))
    model.add(Activation('relu'))
    model.add(Convolution2D(nb_filters, nb_conv, nb_conv))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    
    model.add(Dense(Dense_Before_Out,))
    model.add(Activation('tanh'))
    
    model.add(Dropout(0.25))
    model.add(Dense(Dense_Before_Out,))
    model.add(Activation('tanh'))
    
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
    #sgd = SGD(lr=1e-2, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])
    return model

# Hyper-parameter Configuration

In [22]:

batch_size = 32
nb_classes = 10
nb_epoch = 150
data_augmentation = True

# input image dimensions
img_rows, img_cols = 32, 32
# the CIFAR10 images are RGB
img_channels = 3

nb_filters = 62
# size of pooling area for max pooling
nb_pool = 2
# convolution kernel size
nb_conv = 3

# Loading the data 

In [23]:

# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('X_train shape:', X_train.shape)
print('Y_train shape:', y_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

X_train shape: (50000L, 3L, 32L, 32L)
Y_train shape: (50000L, 1L)
50000 train samples
10000 test samples


In [25]:
#%%capture
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,verbose=1, validation_data=(X_test, Y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/150
   64/50000 [..............................] - ETA: 659294s - loss: 2.6289 - acc: 0.0938

KeyboardInterrupt: 

In [24]:
# Structures

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_5 (Convolution2D)  (None, 62, 30, 30)    1736        convolution2d_input_3[0][0]      
____________________________________________________________________________________________________
activation_11 (Activation)       (None, 62, 30, 30)    0           convolution2d_5[0][0]            
____________________________________________________________________________________________________
convolution2d_6 (Convolution2D)  (None, 62, 28, 28)    34658       activation_11[0][0]              
____________________________________________________________________________________________________
activation_12 (Activation)       (None, 62, 28, 28)    0           convolution2d_6[0][0]            
___________________________________________________________________________________________

In [10]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

KeyboardInterrupt: 

In [ ]:
model_name= str(datetime.now().split(' ')[0])



# serialize model to JSON
model_json = model.to_json()
with open("../hub/model/model{}.json".format(model_name), "w") as json_file:
    json_file.write(model_json)
    
# Saving wreights with the model 
model.save_weights('../hub/model/{}.h5'.format(model_name))

In [1]:
from ipywidgets import interact, interactive, fixed
from ipywidgets import widgets

def f(x):
    
    plt.imshow(Tr_X[x].reshape(img_rows,img_rows,3)) # color
    #plt.imshow(X_train[x].reshape(img_rows,img_rows)) # gray
    print (loaded_model.predict(X_train[x].reshape(1,img_channels,img_rows,img_rows)) )
    print (loaded_model.predict_classes(X_train[x].reshape(1,img_channels,img_rows,img_rows)) ,'--><--', np.argmax(Y_train[x]) )

interact(f,x = 20)

NameError: global name 'plt' is not defined

<function __main__.f>